In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
pip install fastprogress==0.1.20

In [ ]:
cd /content/drive/MyDrive/Team_Singularity/prog

/content/drive/MyDrive/vietai_bvk/prog


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

['vietai-advance-course-retinal-disease-detection', 'cleandata-vietai-caothang']


In [ ]:
# Download image and class labels
#!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/img.jpg
#!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/labels_map.txt
# Get EfficientNet PyTorch
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=7b93e1a4c1ede76f643798a9781feb80440dae98f29a5962f95573ad3300f245
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
import sys

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import optim

import torchvision.transforms as transforms
import torchvision

from fastprogress import master_bar, progress_bar

from PIL import Image

In [ ]:
data = pd.read_csv("../input/vietai-advance-course-retinal-disease-detection/train.csv")
data.head()

,filename,opacity,diabetic retinopathy,glaucoma,macular edema,macular degeneration,retinal vascular occlusion,normal
0,c24a1b14d253.jpg,0,0,0,0,0,1,0
1,9ee905a41651.jpg,0,0,0,0,0,1,0
2,3f58d128caf6.jpg,0,0,1,0,0,0,0
3,4ce6599e7b20.jpg,1,0,0,0,1,0,0
4,0def470360e4.jpg,1,0,0,0,1,0,0


In [ ]:
for label in data.columns[1:]:
    print("Distribution of", label)
    print(data[label].value_counts())
  

Distribution of opacity
0    1902
1    1533
Name: opacity, dtype: int64
Distribution of diabetic retinopathy
0    2680
1     755
Name: diabetic retinopathy, dtype: int64
Distribution of glaucoma
0    2838
1     597
Name: glaucoma, dtype: int64
Distribution of macular edema
0    2919
1     516
Name: macular edema, dtype: int64
Distribution of macular degeneration
0    2861
1     574
Name: macular degeneration, dtype: int64
Distribution of retinal vascular occlusion
0    2995
1     440
Name: retinal vascular occlusion, dtype: int64
Distribution of normal
0    2910
1     525
Name: normal, dtype: int64


In [ ]:
LABELS = data.columns[1:]
def build_label(row):
    return ",".join([LABELS[idx] for idx, val in enumerate(row[1:]) if val == 1])
        
data.apply(lambda x: build_label(x), axis=1).value_counts()

opacity                                                                  671
normal                                                                   525
glaucoma                                                                 327
macular degeneration                                                     299
diabetic retinopathy                                                     212
opacity,macular degeneration                                             212
opacity,diabetic retinopathy                                             186
retinal vascular occlusion                                               170
diabetic retinopathy,macular edema                                       162
opacity,diabetic retinopathy,macular edema                               154
opacity,glaucoma                                                         140
macular edema,retinal vascular occlusion                                  74
opacity,macular edema,retinal vascular occlusion                          62

In [ ]:
len(LABELS)

7

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=2019)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
IMAGE_SIZE = 528                              # Image size (528*528)
IMAGENET_MEAN = [0.485, 0.456, 0.406]         # Mean of ImageNet dataset (used for normalization)
IMAGENET_STD = [0.229, 0.224, 0.225]          # Std of ImageNet dataset (used for normalization)
BATCH_SIZE = 2                           
LEARNING_RATE = 0.001
LEARNING_RATE_SCHEDULE_FACTOR = 0.1           # Parameter used for reducing learning rate
LEARNING_RATE_SCHEDULE_PATIENCE = 4           # Parameter used for reducing learning rate
MAX_EPOCHS = 5                              # Maximum number of training epochs


############
USE_BCELOGIT = True
USE_NOTNORMAL_P = False
USE_AUGMENTATION = True
USE_WEIGHT = True
MIXUP = True

# LOSS WEIGHT
pos_w = None
w = None
if USE_WEIGHT:
    pos_w=[1.3, 1.6, 4, 5.6, 5.2, 6, 5.6]
    w = [3., 1, 1, 1, 1,1, 1]

In [ ]:
!pip install albumentations==0.4.6 > /dev/null

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, Blur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, Resize,  Normalize
)
# GaussianBlur, removed
from albumentations.pytorch import ToTensor
train_aug = Compose([
    Resize(IMAGE_SIZE, IMAGE_SIZE),
    ShiftScaleRotate(shift_limit=(-0.3, 0.3), scale_limit=0.3, rotate_limit=30, p=0.6),
    HorizontalFlip(p=0.5),
    RandomRotate90(),
    Flip(),
    Transpose(),
    OneOf([
        IAAAdditiveGaussianNoise(),
        GaussNoise(),
    ], p=0.2),
    OneOf([
        MotionBlur(p=.2),
        MedianBlur(blur_limit=3, p=0.1),
        Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    OneOf([
        OpticalDistortion(p=0.3),
        GridDistortion(p=.1),        
    ], p=0.2),
    OneOf([
        CLAHE(clip_limit=2),
        IAASharpen(),
        IAAEmboss(),
        RandomBrightnessContrast(),            
    ], p=0.3),
    HueSaturationValue(p=0.3),
    Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ToTensor()
    
])
val_aug = Compose([
    Resize(IMAGE_SIZE, IMAGE_SIZE),
    Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ToTensor()

])

In [ ]:
class FundusDataset(Dataset):
    
    def __init__(self, folder_dir, dataframe, image_size, normalization, is_train=False):
        """
        Init Dataset
        
        Parameters
        ----------
        folder_dir: str
            folder contains all images
        dataframe: pandas.DataFrame
            dataframe contains all information of images
        image_size: int
            image size to rescale
        normalization: bool
            whether applying normalization with mean and std from ImageNet or not
        """
        self.image_paths = [] # List of image paths
        self.image_labels = [] # List of image labels
        self.is_train = is_train
        # Define list of image transformations
        image_transformation = [
            transforms.Resize((image_size, image_size))
        ]
        
        if is_train:
            image_transformation.append(
                torchvision.transforms.RandomAffine(30, translate=(0.1, 0.2), scale=(0.8, 1.2))
            )
            image_transformation.append(
                torchvision.transforms.RandomHorizontalFlip(p=0.5)
            )
        
        image_transformation.append(transforms.ToTensor())
        if normalization:
            # Normalization with mean and std from ImageNet
            image_transformation.append(transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD))
        
        #self.image_transformation = transforms.Compose(image_transformation)
        
        if self.is_train:
            self.image_transformation = train_aug
        else:
            self.image_transformation = val_aug
        
        # Get all image paths and image labels from dataframe
        for index, row in dataframe.iterrows():
            image_path = os.path.join(folder_dir, row.filename)
            self.image_paths.append(image_path)
            self.image_labels.append(row[1:])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        """
        Read image at index and convert to torch Tensor
        """
        
        # Read image
        image_path = self.image_paths[index]
        image_data = Image.open(image_path).convert("RGB") # Convert image to RGB channels
        
        # TODO: Image augmentation code would be placed here

        # Resize and convert image to torch tensor 
        #image_data = self.image_transformation(image_data)
        
        image_data = np.array(image_data)
        image_data = self.image_transformation(image=image_data)['image']
        
        return image_data, torch.FloatTensor(self.image_labels[index])


In [ ]:
train_dataset = FundusDataset("../input/cleandata-vietai-caothang/train", train_data, IMAGE_SIZE, True, USE_AUGMENTATION)

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
for data, label in train_dataloader:
    print(data.size())
    print(label.size())
    break

torch.Size([2, 3, 528, 528])
torch.Size([2, 7])


In [ ]:
val_dataset = FundusDataset("../input/cleandata-vietai-caothang/train", val_data, IMAGE_SIZE, True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
class ResNet50(nn.Module):
    def __init__(self, num_classes, is_trained=True):
        """
        Init model architecture
        
        Parameters
        ----------
        num_classes: int
            number of classes
        is_trained: bool
            whether using pretrained model from ImageNet or not
        """
        super().__init__()
        
        # Load the resnet50 from ImageNet
        self.net = torchvision.models.resnet50(pretrained=is_trained)
        
        # Get the input dimension of last layer
        kernel_count = self.net.fc.in_features
        
        # Replace last layer with new layer that have num_classes nodes, after that apply Sigmoid to the output
        if not USE_BCELOGIT:
            self.net.fc = nn.Sequential(nn.Linear(kernel_count, num_classes)
                                       ,nn.Sigmoid()
                                       )
        else:
            self.net.fc = nn.Sequential(nn.Linear(kernel_count, num_classes)
                                       #,nn.Sigmoid()
                                       )
        
    def forward(self, inputs):
        """
        Forward the netword with the inputs
        """
        output = self.net(inputs)
        if USE_NOTNORMAL_P:
            not_normal_p = 1 - output[:, 0:1]
            output = torch.cat([output[:, 0:1], output[:, 1:] * not_normal_p], dim=1)
        return output


In [ ]:
## Efficient net
from efficientnet_pytorch import EfficientNet
model_name = 'efficientnet-b6'

class EffNet(nn.Module):
    def __init__(self, num_classes, is_trained=True):
        """
        Init model architecture
        
        Parameters
        ----------
        num_classes: int
            number of classes
        is_trained: bool
            whether using pretrained model from ImageNet or not
        """
        super().__init__()
        
        # Load the resnet50 from ImageNet
        self.net = EfficientNet.from_pretrained(model_name)

        
        # Get the input dimension of last layer
        kernel_count = self.net._fc.in_features
        
        # Replace last layer with new layer that have num_classes nodes, after that apply Sigmoid to the output
        if not USE_BCELOGIT:
            self.net._fc = nn.Sequential(nn.Linear(kernel_count, num_classes)
                                       ,nn.Sigmoid()
                                       )
        else:
            self.net._fc = nn.Sequential(nn.Linear(kernel_count, num_classes)
                                       #,nn.Sigmoid()
                                       )
        
    def forward(self, inputs):
        """
        Forward the netword with the inputs
        """
        output = self.net(inputs)
        if USE_NOTNORMAL_P:
            not_normal_p = 1 - output[:, 0:1]
            output = torch.cat([output[:, 0:1], output[:, 1:] * not_normal_p], dim=1)
        return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
model = EffNet(num_classes=len(LABELS)).to(device)
model

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth


  0%|          | 0.00/165M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b6


EffNet(
  (net): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 56, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          56, 56, kernel_size=(3, 3), stride=[1, 1], groups=56, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          56, 14, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          14, 56, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

40751839

In [ ]:
 # Loss function
if pos_w is not None:
    pos_w = torch.tensor(pos_w, device=device)
if w is not None:
    w = torch.tensor(w, device=device)

if USE_BCELOGIT:
    loss_criteria = nn.BCEWithLogitsLoss(weight=w, pos_weight=pos_w)
else:
    loss_criteria = nn.BCELoss(weight=w)
if device == 'cuda':
    loss_criteria.cuda()


# Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-5)

# Learning rate will be reduced automatically during training
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = LEARNING_RATE_SCHEDULE_FACTOR, patience = LEARNING_RATE_SCHEDULE_PATIENCE, mode = 'max', verbose=True)


In [ ]:
def multi_label_f1(y_gt, y_pred, threshold=0.5):
    """ Calculate F1 for each class

    Parameters
    ----------
    y_gt: torch.Tensor
        groundtruth
    y_pred: torch.Tensor
        prediction

    Returns
    -------
    list
        F1 of each class
    """
    f1_out = []
    gt_np = y_gt.to("cpu").numpy()
    pred_np = (y_pred.to("cpu").numpy() > threshold) * 1.0
    assert gt_np.shape == pred_np.shape, "y_gt and y_pred should have the same size"
    for i in range(gt_np.shape[1]):
        f1_out.append(f1_score(gt_np[:, i], pred_np[:, i]))
    return f1_out


In [ ]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def epoch_training(epoch, model, train_dataloader, device, loss_criteria, optimizer, mb, augment=True):
    """
    Epoch training

    Paramteters
    -----------
    epoch: int
      epoch number
    model: torch Module
      model to train
    train_dataloader: Dataset
      data loader for training
    device: str
      "cpu" or "cuda"
    loss_criteria: loss function
      loss function used for training
    optimizer: torch optimizer
      optimizer used for training
    mb: master bar of fastprogress
      progress to log

    Returns
    -------
    float
      training loss
    """
    # Switch model to training mode
    model.train()
    training_loss = 0 # Storing sum of training losses
   
    # For each batch
    for batch, (images, labels) in enumerate(progress_bar(train_dataloader, parent=mb)):            

        # Move X, Y  to device (GPU)
        images = images.to(device)
        labels = labels.to(device)
        if MIXUP:
            images, labels_a, labels_b, lam = mixup_data(images, labels)
        # Clear previous gradient
        optimizer.zero_grad()

        # Feed forward the model
        pred = model(images)
        if MIXUP:
            loss = mixup_criterion(loss_criteria, pred, labels_a, labels_b, lam)
        else:
            loss = loss_criteria(pred, labels)
        # Back propagation
        loss.backward()

        # Update parameters
        optimizer.step()

        # Update training loss after each batch
        training_loss += loss.item()

        mb.child.comment = f'Training loss {training_loss/(batch+1)}'

    del images, labels, loss
    if torch.cuda.is_available(): torch.cuda.empty_cache()

    # return training loss
    return training_loss/len(train_dataloader)


In [ ]:
def evaluating(epoch, model, val_loader, device, loss_criteria, mb):
    """
    Validate model on validation dataset
    
    Parameters
    ----------
    epoch: int
        epoch number
    model: torch Module
        model used for validation
    val_loader: Dataset
        data loader of validation set
    device: str
        "cuda" or "cpu"
    loss_criteria: loss function
      loss function used for training
    mb: master bar of fastprogress
      progress to log
  
    Returns
    -------
    float
        loss on validation set
    float
        metric score on validation set
    """

    # Switch model to evaluation mode
    model.eval()

    val_loss = 0                                   # Total loss of model on validation set
    out_pred = torch.FloatTensor().to(device)      # Tensor stores prediction values
    out_gt = torch.FloatTensor().to(device)        # Tensor stores groundtruth values

    with torch.no_grad(): # Turn off gradient
        # For each batch
        for step, (images, labels) in enumerate(progress_bar(val_loader, parent=mb)):
            # Move images, labels to device (GPU)
            images = images.to(device)
            labels = labels.to(device)

            # Update groundtruth values
            out_gt = torch.cat((out_gt,  labels), 0)

            # Feed forward the model
            ps = model(images)
            loss = loss_criteria(ps, labels)

            # Update prediction values
            out_pred = torch.cat((out_pred, ps), 0)
            
            
            # Update validation loss after each batch
            val_loss += loss
            mb.child.comment = f'Validation loss {val_loss/(step+1)}'

    # Clear memory
    del images, labels, loss
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    # return validation loss, and metric score
    return val_loss/len(val_loader), np.array(multi_label_f1(out_gt, out_pred)).mean()

In [ ]:
#!pip install fastprogress==0.1.20 ## dont use now

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
pip install fastprogress==0.1.20

In [ ]:
################################################################################################
# Dont run this if you are running inference or only testing
################################################################################################


# Best F1 value during training
best_score = 0
best_loss = 10
model_path = "resnet50.pth"
training_losses = []
validation_losses = []
validation_score = []


# Config progress bar
mb = master_bar(range(MAX_EPOCHS))
mb.names = ['Training loss', 'Validation loss', 'Validation F1']
x = []

# Training each epoch
for epoch in mb:
    mb.first_bar.comment = f'Best F1 score: {best_score}'
    x.append(epoch)

    # Training
    model.load_state_dict(torch.load(model_path))
    train_loss = epoch_training(epoch, model, train_dataloader, device, loss_criteria, optimizer, mb)
    mb.write('Finish training epoch {} with loss {:.4f}'.format(epoch, train_loss))
    training_losses.append(train_loss)

    # Evaluating
    val_loss, new_score = evaluating(epoch, model, val_dataloader, device, loss_criteria, mb)
    mb.write('Finish validation epoch {} with loss {:.4f} and score {:.4f}'.format(epoch, val_loss, new_score))
    validation_losses.append(val_loss)
    validation_score.append(new_score)

    # Update learning rate
    lr_scheduler.step(new_score)

    # Update training chart
    mb.update_graph([[x, training_losses], [x, validation_losses], [x, validation_score]], [0,MAX_EPOCHS], [0,1])

    # Save model
    if best_score < new_score:
        mb.write(f"Improve F1 from {best_score} to {new_score}")
        best_score = new_score
        best_loss = val_loss
        # Saving model: https://pytorch.org/tutorials/beginner/saving_loading_models.html
        torch.save(model.state_dict(), model_path)
    if new_score == 1.0 and best_loss > val_loss:
        mb.write(f"Improve score from {best_loss} to {val_loss}")
        best_score = new_score
        best_loss = val_loss
        # Saving model: https://pytorch.org/tutorials/beginner/saving_loading_models.html
        torch.save(model.state_dict(), model_path)

KeyboardInterrupt: ignored

In [ ]:

model_path = "resnet50 (2).pth"
model.load_state_dict(torch.load(model_path))
model.eval()
def eval_thres(model, val_loader, threshold=0.5):
    """
    Validate model on validation dataset
    
    Parameters
    ----------
    epoch: int
        epoch number
    model: torch Module
        model used for validation
    val_loader: Dataset
        data loader of validation set
    device: str
        "cuda" or "cpu"
    loss_criteria: loss function
      loss function used for training
    mb: master bar of fastprogress
      progress to log
  
    Returns
    -------
    float
        loss on validation set
    float
        metric score on validation set
    """

    # Switch model to evaluation mode
    model.eval()

    val_loss = 0                                   # Total loss of model on validation set
    out_pred = torch.FloatTensor().to(device)      # Tensor stores prediction values
    out_gt = torch.FloatTensor().to(device)        # Tensor stores groundtruth values

    with torch.no_grad(): # Turn off gradient
        # For each batch
        for step, (images, labels) in enumerate(progress_bar(val_loader, parent=mb)):
            # Move images, labels to device (GPU)
            images = images.to(device)
            labels = labels.to(device)

            # Update groundtruth values
            out_gt = torch.cat((out_gt,  labels), 0)

            # Feed forward the model
            ps = model(images)

            # Update prediction values
            out_pred = torch.cat((out_pred, ps), 0)
            

    # Clear memory
    del images, labels
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    # return validation loss, and metric score
    return np.array(multi_label_f1(out_gt, out_pred, threshold)).mean()

best_score, best_threshold = 0, 0
for threshold in [0.2,0.25,0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]:
    score = eval_thres(model, val_dataloader, threshold)
    if score > best_score:
        best_score = score
        best_threshold = threshold
    print(f'{threshold} - {score}')

0.2 - 0.8091223618918801
0.25 - 0.8092990778061182
0.3 - 0.8106005251343378
0.35 - 0.8129352501421377
0.4 - 0.8139571863823066
0.45 - 0.812918515284038
0.5 - 0.8145617346293997
0.55 - 0.815044740237545
0.6 - 0.8149821968023272


In [ ]:
test_df = pd.read_csv("../input/vietai-advance-course-retinal-disease-detection/sample_submission.csv")
test_df.head()

,filename,predicted
0,e742d34a26d4.jpg,0 0
1,b193b6a6d68d.jpg,0 0
2,07e4191fa3a8.jpg,0 0
3,b1a911cb2e6c.jpg,0 0
4,d8ab9cda1b33.jpg,0 0


In [ ]:
from torch.nn import functional as F
# Define list of image transformations
image_transformation = [
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor()
]
# Normalization with mean and std from ImageNet
image_transformation.append(transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD))
image_transformation = transforms.Compose(image_transformation)

def predict(image_path, model, device):
    """ Predict image
    
    Parameters
    ----------
    image_path: str
        image path to predict
    model: nn.Module
        model used to predict
    device: str
        'cpu' or 'cuda'
        
    Returns
    -------
    str
        list of label indices
    """
    global image_transformation
    
    # Read image
    image_data = Image.open(image_path).convert('RGB')
    image_data = image_transformation(image_data)
    
    predicted_label = []
    with torch.no_grad():
        ps = model(image_data.unsqueeze(0).to(device))
        if USE_BCELOGIT:
            ps = F.sigmoid(ps)
        ps = ps[0]
        for i in range(ps.size()[0]):
            if ps[i].item() > best_threshold: # Threshold is 0.5
                predicted_label.append(i)
        if len(predicted_label) == 0:
            predicted_label.append(0)
    return " ".join([str(label) for label in predicted_label])

In [ ]:
model.load_state_dict(torch.load(model_path))
model.eval()

EffNet(
  (net): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 56, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          56, 56, kernel_size=(3, 3), stride=[1, 1], groups=56, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          56, 14, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          14, 56, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project

In [ ]:
test_df['predicted'] = test_df.filename.map(lambda x: predict(os.path.join("../input/cleandata-vietai-caothang/test",x), model, device))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
# 0 4 in second column indicates diseaase no 0 and 4th one are classified as Positives and remaining are Negative
test_df.head()

,filename,predicted
0,e742d34a26d4.jpg,6
1,b193b6a6d68d.jpg,0 4
2,07e4191fa3a8.jpg,0
3,b1a911cb2e6c.jpg,0
4,d8ab9cda1b33.jpg,0 2 5


In [ ]:
test_df.to_csv("submission.csv", index=False)

In [ ]:
print(model_path)

resnet50 (2).pth
